```
################################################################################
# Copyright (c) 2022 Bayerische Motoren Werke (BMW) AG
#
# This program and the accompanying materials are made available under the
# terms of the Eclipse Public License 2.0 which is available at
# http://www.eclipse.org/legal/epl-2.0.
#
# SPDX-License-Identifier: EPL-2.0
################################################################################
```

In [1]:
import pandas as pd
import re

In [6]:
events = pd.read_csv('events.csv')
# Available events
events

,RunId,Timestep,Source,Name,AgentId,IsTriggering,IsAffected,Key,Value
0,0,0,OpenSCENARIO,StateChangeStory/Act1/StateChangeSequenceA/Sta...,0,False,True,NaN,NaN
1,0,0,Dynamics_TrajectoryFollower,Dynamics_TrajectoryFollower,0,True,False,ComponentState,Acting
2,0,0,Dynamics_TrajectoryFollower,Dynamics_TrajectoryFollower,0,True,False,InternalState,TrajectoryFollowerActivated


In [5]:
# The event, we are intersted in
event_name = 'Dynamics_TrajectoryFollower'

In [13]:
def normalize_column(column, event):
    # already prefixed
    normalized_column = f'{event}_' + re.sub(f'^({event}_?)', '', column)
    # special chars
    normalized_column = re.sub(r'[^a-zA-Z1-9_]', '_', normalized_column)
    # duplicate _
    normalized_column = re.sub(r'([_])\1+', r'\1', normalized_column)
    # leading and trailing _
    normalized_column = re.sub(r'^_|_$', '', normalized_column)
    return normalized_column

def normalize(event):
    return normalize_column(event, event)



In [8]:
# subset all events of specific type
df = events[events.Name == event_name].copy()
df

,RunId,Timestep,Source,Name,AgentId,IsTriggering,IsAffected,Key,Value
1,0,0,Dynamics_TrajectoryFollower,Dynamics_TrajectoryFollower,0,True,False,ComponentState,Acting
2,0,0,Dynamics_TrajectoryFollower,Dynamics_TrajectoryFollower,0,True,False,InternalState,TrajectoryFollowerActivated


In [9]:
# store current columns except the one dropped in the next step
columns = df.columns.drop(['Key', 'Value'])

# Merge Key/Value columns into a single Column
df_tmp = pd.pivot_table(
    df, values='Value', index=df.index, columns='Key', aggfunc='first')

df_tmp

Key,ComponentState,InternalState
1,Acting,NaN
2,NaN,TrajectoryFollowerActivated


In [10]:
# Concat with original events
df = pd.concat([df[columns], df_tmp], axis='columns')
df

,RunId,Timestep,Source,Name,AgentId,IsTriggering,IsAffected,ComponentState,InternalState
1,0,0,Dynamics_TrajectoryFollower,Dynamics_TrajectoryFollower,0,True,False,Acting,NaN
2,0,0,Dynamics_TrajectoryFollower,Dynamics_TrajectoryFollower,0,True,False,NaN,TrajectoryFollowerActivated


In [12]:
# New columns (= Key) get prefix 'Event_Name_' and are properly normalized
new_columns = df.columns.drop(columns)
mapping = dict()
for new_column in new_columns:
    mapping.update(
        {new_column: f'Event_{normalize_column(new_column, event_name)}'})
df.rename(columns=mapping, inplace=True)
df

,RunId,Timestep,Source,Name,AgentId,IsTriggering,IsAffected,Event_Dynamics_TrajectoryFollower_Event_Dynamics_TrajectoryFollower_ComponentState,Event_Dynamics_TrajectoryFollower_Event_Dynamics_TrajectoryFollower_InternalState
1,0,0,Dynamics_TrajectoryFollower,Dynamics_TrajectoryFollower,0,True,False,Acting,NaN
2,0,0,Dynamics_TrajectoryFollower,Dynamics_TrajectoryFollower,0,True,False,NaN,TrajectoryFollowerActivated


In [14]:
# Add Event_Name to dataframe
df[f'Event_{normalize(event_name)}'] = True
df

,RunId,Timestep,Source,Name,AgentId,IsTriggering,IsAffected,Event_Dynamics_TrajectoryFollower_Event_Dynamics_TrajectoryFollower_ComponentState,Event_Dynamics_TrajectoryFollower_Event_Dynamics_TrajectoryFollower_InternalState,Event_Dynamics_TrajectoryFollower
1,0,0,Dynamics_TrajectoryFollower,Dynamics_TrajectoryFollower,0,True,False,Acting,NaN,True
2,0,0,Dynamics_TrajectoryFollower,Dynamics_TrajectoryFollower,0,True,False,NaN,TrajectoryFollowerActivated,True
